In [1]:
%load_ext autoreload
%autoreload 2
import gym
import os
import numpy as np
import random
import torch
from experiment import rollout, ReplayBuffer, Trajectory, load_model, save_model, Behavior
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
env = gym.make('LunarLander-v2')

In [3]:
rb = ReplayBuffer(max_size=500, last_few=200)

# Random rollout
trajectories, avg_reward = rollout(episodes=500, env=env, render=False)
rb.add(trajectories)

print(f"Average Episode Reward: {avg_reward}")

Average Episode Reward: -180.7511269386291


In [4]:
loss_object = torch.nn.CrossEntropyLoss().to(device)
model_sample = Behavior(input_shape=env.observation_space.shape[0]+2, num_actions=env.action_space.n).to(device)
optimizer = torch.optim.Adam(model_sample.parameters())

In [5]:
batch_size = 1024

In [6]:
def train_step(model, inputs, targets):
    optimizer.zero_grad()    
    predictions = model(inputs)
    loss = loss_object(predictions, targets)
    
    loss.backward()
    optimizer.step()
    
    return loss

def action_fn(model, inputs, sample_action=True):
    action_logits = model(inputs)
    action_probs = torch.softmax(action_logits, axis=-1)

    if sample_action:        
        m = torch.distributions.categorical.Categorical(logits=action_logits)             
        action = int(m.sample().squeeze().cpu().numpy())        
    else:
        action = int(np.argmax(action_probs.detach().squeeze().numpy()))
    return action
    

In [7]:
# loss_sum = 0
# loss_count = 0

# epochs = 1000000
# epoch, model, optimizer, loss = load_model('lunar_lander', model, optimizer, device, train=True)

# for i in range(epoch, epochs+epoch):
#     x, y = rb.sample(batch_size, device)    
#     loss = train_step(x, y)
#     loss_sum += loss
#     loss_count += 1
    
#     if i % 100 == 0:        
#         trajectories, mean_reward = rollout(100, env=env, model=model, sample_action=False, replay_buffer=rb, 
#                               device=device, action_fn=action_fn)
#         rb.add(trajectories)
        
#         print(f"Average Episode Reward: {mean_reward}")        

#     if i % 200 == 0:
#         print(f'i: {i}, Loss: {loss_sum/loss_count}') #'\t Accuracy: {accuracy_m.result()}')
#         save_model('lunar_lander', i, model, optimizer, loss)

In [ ]:
# SAMPLE ACTIONS

loss_sum = 0
loss_count = 0

epochs = 1000000
epoch, model_sample, optimizer, loss = load_model('lunar_lander_sample_actions', model_sample, optimizer, device, train=True)

for i in range(epoch, epochs+epoch):
    x, y = rb.sample(batch_size, device)    
    loss = train_step(model_sample, x, y)
    loss_sum += loss
    loss_count += 1
    
    if i % 500 == 0:        
        trajectories, mean_reward = rollout(100, env=env, model=model_sample, sample_action=True, replay_buffer=rb, 
                              device=device, action_fn=action_fn)
        rb.add(trajectories)
        
        print(f"Average Episode Reward: {mean_reward}")        

    if i % 200 == 0:
        avg_loss = loss_sum/loss_count
        print(f'i: {i}, Loss: {avg_loss}') #'\t Accuracy: {accuracy_m.result()}')
        save_model('lunar_lander_sample_actions', i, model_sample, optimizer, avg_loss)

Existing model found. Loading from epoch 800 with loss: 1.3336138725280762
i: 800, Loss: 1.3301100730895996


In [10]:
rb.sample_command()

(85.67, -66.2490563762739)

In [10]:
cmd = (280.085, 318.91295076177306)
rb.sample_command()
env = gym.make('LunarLander-v2')
e, model, _, l = load_model(name='lunar_lander_sample_actions', train=False, model=model_sample, optimizer=optimizer, device=device)

# _, mean_reward = rollout(episodes=1, env=env, model=model, sample_action=False, 
#                       replay_buffer=rb, render=True, device=device, action_fn=action_fn)
_, mean_reward = rollout(episodes=1, env=env, model=model_sample, sample_action=False, 
                      cmd=cmd, render=True, device=device, action_fn=action_fn)


print(f"Average Episode Reward: {mean_reward}")

Existing model found. Loading from epoch 0 with loss: 1.5534955263137817
Average Episode Reward: -637.9971553073198
